In [1]:
!pip install pm4py 

     |████████████████████████████████| 1.5 MB 5.2 MB/s 
     |████████████████████████████████| 40.6 MB 70 kB/s 
  Created wheel for stringdist: filename=StringDist-1.0.9-cp37-cp37m-linux_x86_64.whl size=23594 sha256=ede8d132abf405a60dbbf9367204e86fcfb0c2171e9285756da2d39ae1c1a905
  Stored in directory: /root/.cache/pip/wheels/d7/9c/d4/63bc3d8931de0980b9e4a724dea290bb40b8b1b2bd6227c8da
Successfully built stringdist


In [2]:
import pandas as pd
import numpy as np

from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.algo.discovery.dfg import algorithm as dfg_fac
from pm4py.objects.conversion.log import converter as log_converter
from pm4py.objects.log.util import dataframe_utils

from tqdm import tqdm

In [3]:
!wget http://icpmconference.org/2020/wp-content/uploads/sites/4/2020/03/DomesticDeclarations.xes_.gz

--2021-08-25 18:58:09--  http://icpmconference.org/2020/wp-content/uploads/sites/4/2020/03/DomesticDeclarations.xes_.gz
Resolving icpmconference.org (icpmconference.org)... 37.128.148.44
Connecting to icpmconference.org (icpmconference.org)|37.128.148.44|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://icpmconference.org/2020/wp-content/uploads/sites/4/2020/03/DomesticDeclarations.xes_.gz [following]
--2021-08-25 18:58:10--  https://icpmconference.org/2020/wp-content/uploads/sites/4/2020/03/DomesticDeclarations.xes_.gz
Connecting to icpmconference.org (icpmconference.org)|37.128.148.44|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 933930 (912K) [application/x-gzip]
Saving to: ‘DomesticDeclarations.xes_.gz’

DomesticDeclaration 100%[===================>] 912.04K  1.79MB/s    in 0.5s    

2021-08-25 18:58:11 (1.79 MB/s) - ‘DomesticDeclarations.xes_.gz’ saved [933930/933930]



In [4]:
!gunzip /content/DomesticDeclarations.xes_.gz

In [5]:
!mv /content/DomesticDeclarations.xes_ /content/DomesticDeclarations.xes

In [6]:
class Transformer:
  def __init__(self, path_to_file):
    self.path = path_to_file
    self.log = None

  def get_from_xes(self):
    self.log = xes_importer.apply(self.path)
    self.dataframe = log_converter.apply(self.log, variant=log_converter.Variants.TO_DATA_FRAME)
    return self.dataframe

  def get_from_csv(self):
    self.dataframe = pd.read_csv(self.path)
    self.dataframe = dataframe_utils.convert_timestamp_columns_in_df(self.dataframe)
    self.log = log_converter.apply(self.dataframe)
    return self.dataframe

  def get_PD_from_log(self):
    df = pd.read_fwf(self.path)
    cols = df.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    self.dataframe = df[cols]
    self.log = log_converter.apply(self.log, variant=log_converter.Variants.TO_EVENT_LOG)
    return self.dataframe

In [7]:
t = Transformer('/content/DomesticDeclarations.xes')
t.get_from_xes()  
cases = np.unique(t.dataframe['case:concept:name'])
avarage_duraion = []
for case in tqdm(cases):
    tmp_df = t.dataframe[t.dataframe['case:concept:name']==case]
    avarage_duraion.append((case, max(tmp_df['time:timestamp'])-min(tmp_df['time:timestamp'])))
avarage_duraion = sorted(avarage_duraion, key=lambda tup: tup[1], reverse=True)
bottle_cases = avarage_duraion[:round(len(avarage_duraion)/10)]

parsing log, completed traces ::   0%|          | 0/10500 [00:00<?, ?it/s]

100%|██████████| 10500/10500 [00:43<00:00, 242.47it/s]


In [8]:
cases = list(map(lambda x: x[0], bottle_cases))
tmp_df = t.dataframe[t.dataframe['case:concept:name'].isin(cases)]
tmp_df.index = [i for i in range(tmp_df.shape[0])]

In [9]:
duration_dict = dict()
for i in range(tmp_df.shape[0]-1):
    edge = (tmp_df['concept:name'][i], tmp_df['concept:name'][i+1])
    duration = tmp_df['time:timestamp'][i+1] - tmp_df['time:timestamp'][i]
    if edge not in duration_dict:
        duration_dict[edge] = [duration]
    else:
        duration_dict[edge].append(duration)
for key in duration_dict.keys():
    duration_dict[key] = np.mean(duration_dict[key])

In [10]:
edge_tuple = []
for key in duration_dict.keys():
    edge_tuple.append((key, duration_dict[key]))
edge_tuple = sorted(edge_tuple, key=lambda tup: tup[1], reverse=True)
edge_tuple[:round(len(edge_tuple)/10)]

[(('Declaration FINAL_APPROVED by SUPERVISOR', 'Payment Handled'),
  datetime.timedelta(days=81, seconds=50604, microseconds=571429)),
 (('Declaration FINAL_APPROVED by SUPERVISOR',
   'Declaration REJECTED by MISSING'),
  datetime.timedelta(days=28, seconds=59776, microseconds=35714)),
 (('Request Payment', 'Declaration REJECTED by MISSING'),
  datetime.timedelta(days=28, seconds=30432))]